In [76]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from fastai.dataset import *
from fastai.structured import *
from fastai.column_data import *

from math import ceil

from dateutil import parser

In [77]:
torch.cuda.set_device(0)

RuntimeError: cuda runtime error (35) : CUDA driver version is insufficient for CUDA runtime version at torch/csrc/cuda/Module.cpp:88

In [78]:
import fastai; fastai.__file__

'/home/jeremy/Documents/repositories/Kaggle/competitive-data-science-predict-future-sales/fastai/__init__.py'

In [79]:
PATH = "data/"
sz=224
arch=resnext101_64
batch_size=5

## Exploratory Analysis and Feature Engineering

In [80]:
!ls {PATH}

item_categories.csv  sales_train.csv	       shops.csv
items.csv	     sample_submission.csv     test.csv
models		     sample_submission.csv.gz  test.csv.gz


In [81]:
sample_subm = pd.read_csv(f'{PATH}sample_submission.csv')
display(sample_subm.tail())

,ID,item_cnt_month
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5
214199,214199,0.5


In [82]:
item_categories = pd.read_csv(f'{PATH}item_categories.csv')
items = pd.read_csv(f'{PATH}items.csv')
shops = pd.read_csv(f'{PATH}shops.csv')
train = pd.read_csv(f'{PATH}sales_train.csv')
test = pd.read_csv(f'{PATH}test.csv')

In [83]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [84]:
#Make Monthly
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train = train.drop(['date'], axis=1)
train = train.groupby([c for c in train.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
train = train.rename(columns={'item_cnt_day':'item_cnt_month'})
#Monthly Mean
shop_item_monthly_mean = train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_monthly_mean = shop_item_monthly_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})
#Add Mean Feature
train = pd.merge(train, shop_item_monthly_mean, how='left', on=['shop_id','item_id'])
#Last Month (Oct 2015)
shop_item_prev_month = train[train['date_block_num']==33][['shop_id','item_id','item_cnt_month']]
shop_item_prev_month = shop_item_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})
shop_item_prev_month.head()
#Add Previous Month Feature
train = pd.merge(train, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
train = pd.merge(train, items, how='left', on='item_id')
#Item Category features
train = pd.merge(train, item_categories, how='left', on='item_category_id')
#Shops features
train = pd.merge(train, shops, how='left', on='shop_id')
train.head()

,date_block_num,shop_id,item_id,item_price,month,year,item_cnt_month,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,221.0,1,2013,6.0,8.000000,0.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,0,33,347.0,1,2013,3.0,3.000000,0.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
2,0,0,35,247.0,1,2013,1.0,7.500000,0.0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
3,0,0,43,221.0,1,2013,1.0,1.000000,0.0,100 МИЛЛИОНОВ ЕВРО,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
4,0,0,51,127.0,1,2013,1.0,1.666667,0.0,100 лучших произведений классики (mp3-CD) (Dig...,57,Музыка - MP3,"!Якутск Орджоникидзе, 56 фран"


In [85]:
test = test.drop('ID', axis=1)

In [86]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34
#Add Mean Feature
test = pd.merge(test, shop_item_monthly_mean, how='left', on=['shop_id','item_id']).fillna(0.)
#Add Previous Month Feature
test = pd.merge(test, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
test = pd.merge(test, items, how='left', on='item_id')
#Item Category features
test = pd.merge(test, item_categories, how='left', on='item_category_id')
#Shops features
test = pd.merge(test, shops, how='left', on='shop_id')
test['item_cnt_month'] = 0.
test.head()

,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,5,5037,11,2015,34,1.181818,0.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0.0
2,5,5233,11,2015,34,1.428571,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0.0
4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0.0


In [87]:
price = train.reset_index().set_index(
    ['item_id', 'shop_id', 'date_block_num']).sort_index()[['item_price', 'item_cnt_month']]
price.head()

item_price  item_cnt_month
item_id shop_id date_block_num                            
0       54      20                    58.0             1.0
1       55      15                  4490.0             2.0
                18                  4490.0             1.0
                19                  4490.0             1.0
                20                  4490.0             1.0

In [88]:
# Helpers
def closest_date_block(current_day, item_id, shop_id):
    """Find the block_date which is closest to the current_day, given item_id and shop_id. Returns index integer"""
    if (item_id, shop_id) in price.index:
        search_lst = np.array(price.loc[(item_id, shop_id)].index)        
        return search_lst[np.abs(current_day - search_lst).argmin()]
    return -1
                
def closest_price(current_day, item_id, shop_id):
    closest_date = closest_date_block(current_day, item_id, shop_id)
    if closest_date != -1:
        return price.loc[( item_id, shop_id, closest_date )]['item_price'].values[0]
    return np.nan

counter = 0
from IPython.display import clear_output
def closest_price_lambda(x):
    global counter
    if counter % 5000 == 0:
        # hack to display progress, obviously this isn't a
        # robust way to do things :)
        clear_output()
        print("{}/{}".format(counter, len(test)))
    counter += 1
    result = closest_price(34, x.item_id, x.shop_id)
    if counter == len(test):
        clear_output()
        print("DONE")
    return result

In [89]:
closest_prices = test.apply(closest_price_lambda, axis=1)

DONE


In [90]:
test['item_price'] = closest_prices

# Applying Categorical Variables

In [93]:
items['item_category_id'] = items['item_category_id'].astype('category').cat.as_ordered()
items['item_name'] = items['item_name'].astype('category').cat.as_ordered()
items['item_id'] = items['item_id'].astype('category').cat.as_ordered()
item_categories['item_category_name'] = item_categories['item_category_name'].astype('category').cat.as_ordered()
shops['shop_id'] = shops['shop_id'].astype('category').cat.as_ordered()

In [94]:
cat_vars = [
    'date_block_num',
    'item_category_id',
    'item_category_name',
    'item_id',
    'item_name'
    'month',              
    'shop_id',
    'shop_name',
    'year',               
]
contin_vars = [
    'item_cnt_month_mean',
    'item_cnt_prev_month',
    'item_cnt_month',
    'item_price',
]
dep = 'item_cnt_month'

In [95]:
date_block_num_cat = pd.api.types.CategoricalDtype(categories=[i for i in range(35)], ordered=True)
item_category_id_cat = pd.api.types.CategoricalDtype(categories=np.unique(item_categories['item_category_id']), ordered=True)
item_category_name_cat = pd.api.types.CategoricalDtype(categories=np.unique(item_categories['item_category_name']), ordered=True)
item_id_cat = pd.api.types.CategoricalDtype(categories=np.unique(items['item_id']), ordered=True)
item_name_cat = pd.api.types.CategoricalDtype(categories=np.unique(items['item_name']), ordered=True)
month_cat = pd.api.types.CategoricalDtype(categories=np.unique(train['month']), ordered=True)
shop_id_cat = pd.api.types.CategoricalDtype(categories=np.unique(shops['shop_id']), ordered=True)
shop_name_cat = pd.api.types.CategoricalDtype(categories=np.unique(shops['shop_name']), ordered=True)
year_cat = pd.api.types.CategoricalDtype(categories=np.unique(train['year']), ordered=True)

In [98]:
train['date_block_num'] = train['date_block_num'].astype(date_block_num_cat)
train['item_id'] = train['item_id'].astype(item_id_cat)
train['item_category_id'] = train['item_category_id'].astype(item_category_id_cat)
train['item_category_name'] = train['item_category_name'].astype(item_category_name_cat)
train['item_name'] = train['item_name'].astype(item_name_cat)
train['shop_id'] = train['shop_id'].astype(shop_id_cat)
train['shop_name'] = train['shop_name'].astype(shop_name_cat)
train['month'] = train['month'].astype(month_cat)
train['year'] = train['year'].astype(year_cat)

test['date_block_num'] = test['date_block_num'].astype(date_block_num_cat)
test['item_id'] = test['item_id'].astype(item_id_cat)
test['item_category_id'] = test['item_category_id'].astype(item_category_id_cat)
test['item_category_name'] = test['item_category_name'].astype(item_category_name_cat)
test['item_name'] = test['item_name'].astype(item_name_cat)
test['shop_id'] = test['shop_id'].astype(shop_id_cat)
test['shop_name'] = test['shop_name'].astype(shop_name_cat)
test['month'] = test['month'].astype(month_cat)
test['year'] = test['year'].astype(year_cat)

for v in contin_vars:
    train[v] = train[v].astype('float32')
    test[v] = test[v].astype('float32')


In [100]:
assert not train.isnull().any().any()

In [102]:
assert not test.drop('item_price', axis=1).isnull().any().any() # item price can be nan by design

In [103]:
train.to_pickle(PATH + 'train.pkl')
test.to_pickle(PATH + 'test.pkl')

In [104]:
!ls {PATH}

item_categories.csv  sales_train.csv	       shops.csv    test.pkl
items.csv	     sample_submission.csv     test.csv     train.pkl
models		     sample_submission.csv.gz  test.csv.gz


# Training

## _Deep Learning Approach_

In [ ]:
def rmse(y_pred, targ):
    return math.sqrt(((targ - y_pred)**2).mean())

In [ ]:
val = train[train['date_block_num']==33]

In [ ]:
train = train[train['date_block_num']<33]

In [ ]:
df, y, nas, mapper = proc_df(train, dep, do_scale=True)
val_df, val_y, nas, mapper = proc_df(
    val,
    dep,
    do_scale=True,
    mapper=mapper,
    na_dict=nas,
)
test_df, _, nas, mapper = proc_df(
    test,
    dep,
    do_scale=True,
    mapper=mapper,
    na_dict=nas
)

In [ ]:
y_train = np.log1p(y.clip(0.,20.))
y_val = np.log1p(val_y.clip(0.,20.))

In [ ]:
md = ColumnarModelData.from_data_frames(
    path=PATH,
    trn_df=df,
    trn_y=y_train,
    val_df=val_df,
    val_y=y_val,
    cat_flds=cat_vars,
    bs=256,
    test_df=test_df
)

In [ ]:
?ColumnarModelData.from_data_frames

In [ ]:
cat_sz = [(c, len(train[c].cat.categories)+1) for c in cat_vars]

In [ ]:
cat_sz

In [ ]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
emb_szs

In [ ]:
m = md.get_learner(
    emb_szs=emb_szs,
    n_cont=len(df.columns) - len(cat_vars),
    emb_drop=.01,
    out_sz=1,
    szs=[1000, 500],
    drops=[.05, .01],
)

In [ ]:
m.lr_find(1e-6)

In [ ]:
m.sched.plot(1)

In [ ]:
lr = 1e-6
m.fit(lrs=lr, n_cycle=3, metrics=[rmse], cycle_len=1, cycle_mult=2)

In [ ]:
m.fit(lrs=1e-5, n_cycle=3, metrics=[rmse])

## Submit It!

In [ ]:
sample_subm.head()

In [ ]:
reload_test = pd.read_csv(f'{PATH}test.csv')

In [ ]:
type(shops['shop_id'].reset_index())

In [ ]:
shops.dtypes

In [ ]:
apply_cats(reload_test, shops['shop_id'].reset_index())
apply_cats(reload_test, items['item_id'].reset_index())

In [ ]:
copy_test = test.copy()

In [ ]:
reload_test.dtypes

In [ ]:
%%capture
ids_readded = pd.merge(copy_test, reload_test, on=['shop_id', 'item_id'])

In [ ]:
ids_readded.head()

In [ ]:
assert len(np.unique(ids_readded['ID'])) == len(ids_readded)

In [ ]:
len(np.unique(ids_readded['ID']))

In [ ]:
len(ids_readded)

In [ ]:
ids_readded.head()

In [ ]:
ids_readded.head()

In [ ]:
test_df.head()

In [ ]:
result = m.predict(is_test=True)
predictions = np.expm1(result)
display(predictions)

In [ ]:
ids_readded['item_cnt_month'] = predictions

In [ ]:
final_dataframe = ids_readded[['ID', 'item_cnt_month']]

In [ ]:
final_dataframe.head()

In [ ]:
len(final_dataframe)

In [ ]:
len(np.unique(final_dataframe['ID']))

In [ ]:
SUBM = 'sub/'
os.makedirs(SUBM, exist_ok=True)
final_dataframe.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')